In [1]:
from directory_cobveg_load import definir_diretorio 
project_path = definir_diretorio()

import geopandas as gpd
import os

from utils import(
    get_data_diretorio,
    check_crs,
    save_parquet_excel,
)
from utils.downloads import(
    download_malha_geosampa,
)

# Cobertura Vegetal 2020 (GeoSampa)

In [2]:
data_path = get_data_diretorio(project_path)

## Importar dados 

In [3]:
cache_path = os.path.join(
    data_path, 
    'cache'
)
cache_full_path = os.path.join(
        cache_path, 
        'cobertura_vegetal_concat.parquet'
                )
if os.path.exists(
    cache_full_path
):
    print('Carregando arquivo em cache')
    gdf_cobertura_vegetal = gpd.read_parquet(cache_full_path)
else:
    gdf_cobertura_vegetal = download_malha_geosampa(
        'cobertura_vegetal', 
        cache_path, 
        True
    )

http://wfs.geosampa.prefeitura.sp.gov.br/geoserver/geoportal/wfs?version=1.0.0&request=GetFeature&outputFormat=SHAPE-ZIP&typeName=geoportal:cobertura_vegetal&maxFeatures=2000&startIndex=0
cobertura_vegetal_pg1
(2000, 19)
http://wfs.geosampa.prefeitura.sp.gov.br/geoserver/geoportal/wfs?version=1.0.0&request=GetFeature&outputFormat=SHAPE-ZIP&typeName=geoportal:cobertura_vegetal&maxFeatures=2000&startIndex=2000
cobertura_vegetal_pg2
(2000, 19)
http://wfs.geosampa.prefeitura.sp.gov.br/geoserver/geoportal/wfs?version=1.0.0&request=GetFeature&outputFormat=SHAPE-ZIP&typeName=geoportal:cobertura_vegetal&maxFeatures=2000&startIndex=4000
cobertura_vegetal_pg3
(2000, 19)
http://wfs.geosampa.prefeitura.sp.gov.br/geoserver/geoportal/wfs?version=1.0.0&request=GetFeature&outputFormat=SHAPE-ZIP&typeName=geoportal:cobertura_vegetal&maxFeatures=2000&startIndex=6000
cobertura_vegetal_pg4
(2000, 19)
http://wfs.geosampa.prefeitura.sp.gov.br/geoserver/geoportal/wfs?version=1.0.0&request=GetFeature&outputFor

In [4]:
gdf_cobertura_vegetal.shape

(272117, 19)

In [5]:
gdf_cobertura_vegetal.sample(3)

,cd_identif,cd_identi0,cd_control,tx_datum,cd_coorden,cd_coorde0,cd_categor,cd_subcate,cd_subcat0,tx_descric,qt_area_ve,qt_perimet,qt_altura_,qt_desvio_,tx_data_vo,tx_entrega,tx_lote,tx_lote_ve,geometry
1070,41071.0,41202.0,31507.0,"SIRGAS 2000,4 - 23S",335002.92,7395495.54,13.0,0.0,0.0,"Média a alta cobertura arbórea, arbóreo-arbust...",400.75024,116.41623,0.0,0.0,2017,06_2018,003,V1,"POLYGON ((335009.511 7395489.137, 335010.762 7..."
611,230612.0,231698.0,211385.0,"SIRGAS 2000,4 - 23S",357834.96,7402121.54,11.0,0.0,0.0,"Baixa cobertura arbórea, arbóreo-arbustiva e o...",539.79798,217.83236,0.0,0.0,2017,10_2018,010,V1,"POLYGON ((357802.586 7402106.989, 357796.018 7..."
1567,161568.0,161954.0,168689.0,"SIRGAS 2000,4 - 23S",325481.36,7383088.09,13.0,0.0,0.0,"Média a alta cobertura arbórea, arbóreo-arbust...",1282.27309,277.50927,0.0,0.0,2017,09_2018,009,V2,"POLYGON ((325476.664 7383141.525, 325476.479 7..."


## Padronizaçã de colunas

Há uma série de colunas que podem ser importantes ou não. As que eu escolhi apagar são as que constam no dicionário 'drop_cobveg_cols' a seguir:

In [6]:
drop_cobveg_cols ={
    'cd_control',
    'tx_datum',
    'cd_coorden',
    'cd_coorde0',
    'tx_descric',
    'qt_area_ve',
    'qt_perimet',
    'qt_desvio_',
    'tx_data_vo',
    'tx_entrega',
    'tx_lote',
    'tx_lote_ve'    
}

gdf_cobertura_vegetal.drop(
    columns=drop_cobveg_cols,
    axis=1,
    inplace=True
)

In [7]:
rename_cobveg_cols ={
    'cd_identif' : 'cd_cob_veg'
}

gdf_cobertura_vegetal.rename(
    rename_cobveg_cols,
    axis=1,
    inplace=True
)

## Conferir CRS

In [8]:
gdf_cobertura_vegetal = check_crs(gdf_cobertura_vegetal)

## Salvar GDF

In [9]:
save_parquet_excel(
    gdf_cobertura_vegetal, 
    'malha_cobertura_vegetal',
    data_path,
    data_subpath='assets' 
)